<h1 align="center"><font size="5">OVERVIEW</font></h1>
## 1. Introduction
This project is for Hybrid recommedation for talents, we used different combinations of different technologies such as feature representation approachs(N-Grams, Word Embedding) and different learning algorithms(SVM, Logistic regression) to find the best approach for expert recommendaion of LinkedIn user profiles.
![Gambar](https://raw.githubusercontent.com/srirambaskaran/hybrid-movie-recommender-system/master/Architecture.png)

### Definition of Algorithm
Recommender systems are software tools used to generate and provide suggestions for items and other entities to the users by exploiting various strategies. Hybrid recommender systems combine two or more recommendation strategies in different ways to benefit from their complementary advantages.
### Story
In this kernel Data Scientist want to make recommendation for user who want to watch a movie based on rating (minimum content score)

#### Import Some Library

In [1]:
# Scikit Learn
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from recommendation_data import dataset

#minmax
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import sys
import scipy.stats
import scipy.spatial
import random
from sklearn.metrics import mean_squared_error
from math import sqrt
import math
import warnings
warnings.simplefilter("error")

#### Sneak Peak Dataset

In [2]:
dfmovie = pd.read_csv('../Dataset/data_contents.csv')
dfmovie

,Film,Description
0,Ada Apa dengan Cinta 2,Romance ID
1,Aladdin,Romance US
2,Avengers: End Game,Action US
3,Bumi Manusia,rama ID
4,Captain Marvel,Action US
5,Dilan 1991,Romance ID
6,Dua Garis Biru,Drama ID
7,Gundala,Action ID
8,Spiderman: Far From Home,Action US
9,The Lion King,Drama US


In [3]:
usermov = pd.DataFrame(dataset)
usermov.head()

,Hania,Topik Zulkarnain,AhokTemanFirli,franadek,OM INDRA,Mulya,Genjeh,faizah,Dpv,Putrisqiana,...,bunga,Jawaharal,Febi ganteng gak ada obat,Star,Indra Junior,Indra 1991 SM,Damar Teman Firli,jul,luck,Maria O.
Ada Apa dengan Cinta 2,3,0,0,4,3,0,5,3,5,4,...,0,2,4,4,4,0,5,0,3,0
Gundala,5,0,0,4,0,0,4,0,4,3,...,4,4,3,4,0,5,0,3,0,0
Dilan 1991,4,0,0,4,2,0,3,5,4,2,...,4,3,4,0,0,0,0,0,0,0
Bumi Manusia,4,0,0,5,0,0,0,5,0,4,...,0,0,0,0,0,0,0,0,0,0
Dua Garis Biru,4,0,0,3,5,0,0,4,0,3,...,0,5,0,0,4,0,0,3,0,0


## 2. Execute Algorithm

Hybrid recommendation filtering algorithm is mix between content based and collaborative filtering.<br>
Used sklearn TfidfVectorizer, this is summarizes how often a given word appears within a document.

In [4]:
def content_based(person,min_content_score):
    
    k=0
    not_watch=[]
    for i in usermov[person]:
        if i==0:
            not_watch.append(usermov.index[k])
            k=k+1
        else:
            k=k+1
    
    tf = TfidfVectorizer(analyzer='word',
                             ngram_range=(1, 3),
                             min_df=0,
                             stop_words='english')
    tfidf_matrix=tf.fit_transform(dfmovie['Description'])
    cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    new=pd.DataFrame(cosine_similarities,index=dfmovie['Film'],columns=dfmovie['Film'])
    
    final=pd.DataFrame(new[not_watch].mean().sort_values(ascending=False),columns=['Score'])
    final2=final[final.Score>=min_content_score]
    indeks=final2.index
    
    return indeks

Find the correlation between 2 person

In [5]:
def person_correlation(person1, person2):

   # To get both rated items
    both_rated = {}
    for item in dataset[person1]:
        if item in dataset[person2]:
            both_rated[item] = 1

    number_of_ratings = len(both_rated)

    # Checking for ratings in common
    if number_of_ratings == 0:
        return 0

    # Add up all the preferences of each user
    person1_preferences_sum = sum([dataset[person1][item] for item in both_rated])
    person2_preferences_sum = sum([dataset[person2][item] for item in both_rated])

    # Sum up the squares of preferences of each user
    person1_square_preferences_sum = sum([pow(dataset[person1][item],2) for item in both_rated])
    person2_square_preferences_sum = sum([pow(dataset[person2][item],2) for item in both_rated])

    # Sum up the product value of both preferences for each item
    product_sum_of_both_users = sum([dataset[person1][item] * dataset[person2][item] for item in both_rated])

    # Calculate the pearson score
    numerator_value = product_sum_of_both_users - (person1_preferences_sum*person2_preferences_sum/number_of_ratings)
    denominator_value = sqrt((person1_square_preferences_sum - pow(person1_preferences_sum,2)/number_of_ratings) * (person2_square_preferences_sum -pow(person2_preferences_sum,2)/number_of_ratings))

    if denominator_value == 0:
        return 0
    else:
        r = numerator_value / denominator_value
        return r

Function to make recommendation engine to other user based on minimum content score.

In [6]:
def user_recommendations(person,min_content_score):

    # Gets recommendations for a person by using a weighted average of every other user's rankings
    totals = {}
    simSums = {}
    rankings_list =[]
    for other in dataset:
        # don't compare me to myself
        if other == person:
            continue
        sim = person_correlation(person,other)
        #print ">>>>>>>",sim

        # ignore scores of zero or lower
        if sim <=0: 
            continue
        for item in dataset[other]:

            # only score movies i haven't seen yet
            if item not in dataset[person] or dataset[person][item] == 0:

            # Similrity * score
                totals.setdefault(item,0)
                totals[item] += dataset[other][item]* sim
                # sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+= sim

        # Create the normalized list

    rankings = [(total/simSums[item],item) for item,total in totals.items()]
    rankings.sort()
    rankings.reverse()
    # returns the recommended items
    recommendataions_list = [recommend_item for score,recommend_item in rankings]
    

    new_rankings=[]
    for i in rankings:
        if i[1] in content_based(person,min_content_score):
            new_rankings.append(i)
        
    return new_rankings

Function for hybrid filtering algorithm

In [7]:
def hybrid_filtering(person,min_content_score):
    content_based(person,min_content_score)
    return user_recommendations(person,min_content_score)

## 3. Test Hybrid Filtering

In [8]:
hybrid_filtering('Star',0.3)

[(3.260677717870658, 'Spiderman: Far From Home'),
 (1.7772169098228738, 'Dilan 1991')]

<h1 align="center"><font size="5">CONCLUSION</font></h1>
Based on the hybrid recommendation filtering we can suggest movie score of film for user Star with minimum content score which can modify.